This is experimental work towards a build system for the Jupyter notebooks in the "remote" and "local" folders.

In [1]:
!pip install gitpython

In [2]:
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter

from os import listdir,mkdir
from os.path import join,splitext,isdir
from shutil import copy
from git import Repo
from pathlib import Path

In [3]:
repo_dir=Path(r"\Users\paul_000\Documents\GitHub\o2biz")
repo=Repo(repo_dir)

In [4]:
assert not repo.bare

In [5]:
#
# htmlize and copy both return the name of the file they created,  which in turn gets used in
# process_dir
#

def htmlize(src,dest):
    notebook=nbformat.read(src,as_version=4)    
    html_exporter = HTMLExporter()
    (body, resources) = html_exporter.from_notebook_node(notebook)
    dest_file=splitext(dest)[0]+'.html'
    with open(dest_file,'wt',encoding="utf-8") as outfile:
        outfile.write(body)
    return dest_file
        
extensions={
    '.jpg':copy,
    '.png':copy,
    '.ipynb':htmlize
}
    

def process_dir(source_dir,output_dir):
    files=listdir(source_dir)   
    for f in files:
        src=join(source_dir,f)
        dest=join(output_dir,f)
        if isdir(src):
            if not isdir(dest):
                mkdir(dest)
            process_dir(src,dest)
        else:
            fname,ext=splitext(f)
            if ext in extensions:
                op=extensions[ext]
                newfile=Path(op(src,dest)).relative_to(repo_dir)
                repo.git.add(newfile)
            

In [6]:
nb_source=r".."
nb_output=r"\Users\paul_000\Documents\GitHub\o2biz\o2site\src\main\webapp\notebooks"

def process(subname):
    source_dir=join(nb_source,subname)
    output_dir=join(nb_output,subname)
    process_dir(source_dir,output_dir)

In [8]:
process("remote")
#process("local")

In [8]:
#notebook_path=join(source_dir,'Querying DBpedia.ipynb')

In [9]:
#notebook=nbformat.read(notebook_path,as_version=4)

In [10]:
#html_exporter = HTMLExporter()
#(body, resources) = html_exporter.from_notebook_node(notebook)